## Libraries

In [ ]:
import keras
import sklearn
import tensorflow as tf
import numpy as np
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers

## Fashion MNIST dataset

In [ ]:
def data():
    (train, train_l), (test, test_l) = fashion_mnist.load_data()
    train = train.reshape((train.shape[0], 28, 28, 1))
    test = test.reshape((test.shape[0], 28, 28, 1))
    
    # one-hot encoding the labels
    train_l = to_categorical(train_l)
    test_l = to_categorical(test_l)
    
    return train, train_l, test, test_l

train, train_l, test, test_l = data()

### Normalizing data

In [ ]:
def normalize(train, test):
    train_n = train.astype('float32')
    test_n = test.astype('float32')
    
    train_n = train_n / 255.0
    test_n = test_n / 255.0
    
    return train_n, test_n

train_n, test_n = normalize(train, test)

## Deep Learning Model

In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(28, 28, 1)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(28, 28, 1)))
    model.add(Dropout(0.4))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.9995), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

### Training and Evaluating

In [ ]:
def train_and_evaluate_model(train_n, train_l, test_n, test_l):

    model = create_model()
    train_set, val_set, train_set_l, val_set_l = train_test_split(train_n, train_l, test_size=0.1, random_state=1)
        
    history = model.fit(train_set, train_set_l,
                        epochs=500,
                        batch_size=256,
                        validation_data=(val_set, val_set_l), verbose=1)
    _, acc = model.evaluate(test_n, test_l, verbose=0)
    print("Test accuracy: %.3f" % (acc * 100.0))
        
    return acc, history

### Loss and Accuracy curves for Training and Validation

In [ ]:
def learning_curves(history):
    
    pyplot.subplot(1, 2, 1)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='validation')
    pyplot.legend()

    pyplot.subplot(1, 2, 2)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='validation')
    pyplot.legend()
        
    pyplot.show()

In [ ]:
def run_test(train_n, train_l, test_n, test_l):
    accuracy, history = train_and_evaluate_model(train_n, train_l, test, test_l)
    learning_curves(history)

Achieved Test Accuracy: 90.420%

In [ ]:
## If you have GPU available,
## uncomment the line below, and
## indent the line after

# with tf.device('/device:GPU:0'):
run_test(train_n, train_l, test_n, test_l)